In [25]:
import tensorflow as tf
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import numpy as np
import codecs
from itertools import compress
import keras_bert
import tensorflow_hub as hub
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras_bert.layers import MaskedGlobalMaxPool1D,MaskedConv1D
print(tf.__version__)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
# pd.options.display.max_colwidth=100

1.13.1


In [2]:
path='/NR/wd/1.Project/6.Multi-label_with_BERT/factselect.csv'
df_all=pd.read_csv(path)
df_all.head(2)

,text,labels,File_id,ind:1344793_Financial Services,"ind:1345595_Securities, Commodity Contracts and Financial Investments",ind:1344794_Technology,ind:1344780_Healthcare,ind:1345016_Banking and Credit,ind:1345359_Banking,ind:1344788_Pharmaceuticals,...,ind:1345379_Biotechnology,ind:1344797_Energy Industry,ind:1345570_Search Engines,ind:1344801_Specialized Drugs and Medications,ind:1345243_Computer Software,ind:1348232_Computers,ind:1345692_Alternative Investments,ind:1348218_Media Industry,ind:1344990_Exchange-Traded Funds,Num_labels
0,Best s Rating Notification MIC Property and Ca...,ind:1344793_Financial Services,40121673_AMBEST0020180829ee8t001jn_FT_40121673...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,De novos built for niche audiences Albert Amin...,ind:1344898_Commercial Banking | ind:1344793_F...,40121673_AMBO000020180827ee8r000b5_FT_40121673...,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,4


In [3]:
train_df,test_df=train_test_split(df_all,test_size=0.2,random_state=123,stratify=df_all.Num_labels)
train_df.shape,test_df.shape

((174, 34), (44, 34))

In [4]:
target_labels=list(train_df.columns[3:-1])
len(target_labels)

30

In [5]:
y_train=train_df[target_labels].values
y_test=test_df[target_labels].values
y_train.shape,y_test.shape

((174, 30), (44, 30))

In [8]:
X_train_text=train_df.text.values
X_test_text=test_df.text.values
train_features = {
    "news": np.array(X_train_text)
    }
train_labels = np.array(y_train)
test_features = {
    "news": np.array(X_test_text)
    }
test_labels=np.array(y_test)

In [19]:
embedded_text_feature_column = hub.text_embedding_column(
    key="news", 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)

In [20]:
num_classes=len(y_train[0])
print('Number of labels :',num_classes)

multi_label_head = tf.contrib.estimator.multi_label_head(num_classes,loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE)

Number of labels : 30


In [36]:
estimator = tf.estimator.DNNEstimator(head=multi_label_head,
                                      hidden_units=[1024, 512, 256],
                                      feature_columns=[embedded_text_feature_column],
                                      optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\NAFISU~1.RAH\\AppData\\Local\\Temp\\tmppw8761t1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E48815A2E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [37]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(train_features, train_labels, shuffle=True, batch_size=32, num_epochs=25)
test_input_fn = tf.estimator.inputs.numpy_input_fn(test_features, test_labels, shuffle=False)

In [38]:
%%time
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\NAFISU~1.RAH\AppData\Local\Temp\tmppw8761t1\model.ckpt.
INFO:tensorflow:loss = 0.6926433, step = 1
INFO:tensorflow:global_step/sec: 6.22237
INFO:tensorflow:loss = 0.22476986, step = 101 (16.071 sec)
INFO:tensorflow:Saving checkpoints for 136 into C:\Users\NAFISU~1.RAH\AppData\Local\Temp\tmppw8761t1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.16917609.
Wall time: 1min 47s


In [39]:
estimator.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-02T08:09:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\NAFISU~1.RAH\AppData\Local\Temp\tmppw8761t1\model.ckpt-136
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-02-08:10:04
INFO:tensorflow:Saving dict for global step 136: auc = 0.83928716, auc_precision_recall = 0.52075624, average_loss = 0.22397913, global_step = 136, loss = 0.22397913
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 136: C:\Users\NAFISU~1.RAH\AppData\Local\Temp\tmppw8761t1\model.ckpt-136


{'auc': 0.83928716,
 'auc_precision_recall': 0.52075624,
 'average_loss': 0.22397913,
 'loss': 0.22397913,
 'global_step': 136}

In [40]:
i=11
sample_text=[X_test_text[i]]
subject=list(compress(target_labels, y_test[i]))
print(sample_text)
print(subject)

['UPDATE 1 Activist Elliott argues Sky is worth over 34 billion, UK regulator says Updates with detail and background By Ben Martin LONDON, Aug 29 Reuters Activist hedge fund Elliott believes Walt Disney s deal to buy assets from Twenty First Century Fox has valued Sky at more than 34 billion, higher than any of the current offers for the British broadcaster, Britain s takeover regulator disclosed on Wednesday. Sky is at centre of a bidding battle between Rupert Murdoch s Fox, which already owns 39 percent of the pay television company, and U.S. cable giant Comcast. Disney has also agreed a separate deal to buy TV and film assets from Fox, including its Sky shareholding, for about 71 billion. Disney was forced to raise the price it is paying for the Fox businesses in June to 38 a share after Comcast also tried to gate crash that deal. Comcast currently has the upper hand in the fight for Sky after it offered 14.75 pounds a share for the broadcaster last month, trumping a sweetened 14 p

In [41]:
# Generate predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"news": np.array(sample_text)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [42]:
for r in results:
    prob = r['probabilities']
    result_df=pd.DataFrame({'subject_name':target_labels,'prob':prob})
    print(result_df.sort_values(by=['prob'],ascending=False)[0:10])
    print('True Label :',subject)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\NAFISU~1.RAH\AppData\Local\Temp\tmppw8761t1\model.ckpt-136
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
                                         subject_name      prob
0                      ind:1344793_Financial Services  0.874241
1   ind:1345595_Securities, Commodity Contracts an...  0.451231
4                      ind:1345016_Banking and Credit  0.284377
9                   ind:1345727_Financial Investments  0.264354
5                                 ind:1345359_Banking  0.203440
7    ind:1345190_Trusts, Funds and Financial Vehicles  0.144487
29                  ind:1344990_Exchange-Traded Funds  0.122974
14     

In [ ]:
estimator.export_saved_model()